In [1]:
from jupyter_cadquery import set_defaults, set_sidecar
from jupyter_cadquery.cadquery import (PartGroup, Part, show as show_object)

set_defaults(theme="dark")
set_sidecar("CadQuery", init=True)
import cadquery as cq


Overwriting auto display for cadquery Workplane and Shape


/home/semartin/miniconda3/envs/jcq22/lib/python3.8/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [12]:
BOARD_DIM = [26.7, 53.7, 8.6]
THICK = 2
SCREW_HOLE_DIA = 4.2

SCREEN_DIM = [17.8, 27.4, 100]
SCREEN_OFFS_FROM_FRONT = 7
RESET_SW_OFFS = [15.5, 3.5]

screen_offs = (BOARD_DIM[1]-SCREEN_DIM[1])/2 - SCREEN_OFFS_FROM_FRONT
box_dim = [BOARD_DIM[0]+2*THICK, BOARD_DIM[1]+2*THICK, BOARD_DIM[2]+THICK]

board = (
    cq.Workplane().box(*BOARD_DIM, centered=[True, True, False]).translate((0,-screen_offs,0))
    .union(
        cq.Workplane().box(*SCREEN_DIM, centered=[True, True, False])
        .translate((0,0,BOARD_DIM[2]))
    )
    .cut( # Downward "bezel" bordering the screen, to better hide the electronics
        cq.Workplane().rect(*SCREEN_DIM[:2]).rect(SCREEN_DIM[0]+THICK, SCREEN_DIM[1]+THICK)
        .extrude(-THICK/2)
        .translate((0,0,BOARD_DIM[2]))
    )
)


hole_mount = (
    cq.Workplane().box(2*SCREW_HOLE_DIA+2*THICK, 2*SCREW_HOLE_DIA+2*THICK, 2*THICK, centered=[True, True, False])
    .edges(">Y and |Z").fillet(SCREW_HOLE_DIA)
    .faces(">Z").workplane().hole(SCREW_HOLE_DIA)
    .translate((0,-SCREW_HOLE_DIA/2,0))
)

reset_sw_hole = (cq.Workplane("YZ")
                 .center(-BOARD_DIM[1]/2 - SCREEN_OFFS_FROM_FRONT + RESET_SW_OFFS[0], BOARD_DIM[2]-RESET_SW_OFFS[1])
                    .rect(5,3, centered=True).extrude(100))

logo = (
    cq.importers.importDXF('touchmark.dxf')
    .wires().toPending()
    .extrude(1)
    .translate((-8.5,-8.5,)) # Center origin
    .rotate((0,0,0),(0,0,1),90) # Switch orientation
    .translate(( # Move to open spot on cover
        0, 
        -(box_dim[1]/2 - SCREEN_DIM[1]/2 + SCREEN_OFFS_FROM_FRONT + THICK), 
        BOARD_DIM[2]+THICK/2
    ))
)

cover = (
    cq.Workplane().box(*box_dim, centered=[True, True, False])
    .translate((0,-screen_offs, 0))
    .cut(board)
    .cut(reset_sw_hole)
    
    # Prettify face
    .faces(">Z").chamfer(THICK/2)
    .cut(logo)
    
    # Add mounting holes
    .union(hole_mount.translate((0,box_dim[1]/2,0)))
    .union(hole_mount.rotate((0,0,0),(0,0,1),180)
           .translate((0,-box_dim[1]/2-2*screen_offs,0)))
)
cover
# PartGroup([Part(cover), Part(logo), Part(cq.Workplane().sphere(1))])




Done, using side car 'Cadquery'


# TTGO Display Cover

Covers the TTGO T-Display board used in the spincaster control panel

In [ ]:
show_object(button)

In [3]:
from pathlib import Path; downloads = str(Path.home() / "Downloads")
from mkr3d.commands import post
cq.exporters.export(cover, f"{downloads}/spincaster_display_cover.stl")
post(f"{downloads}/spincaster_display_cover.stl", slice=True, queue=True, publish=True)
print("done")

{'exec_dir': '/home/semartin/Documents/devenv/mkr3d/grid-apps'}
/home/semartin/miniconda3/envs/jcq22/lib/python3.8/site-packages/mkr3d/kirimoto.py
node /home/semartin/miniconda3/envs/jcq22/lib/python3.8/site-packages/mkr3d/kiri-cli.js /home/semartin/Documents/devenv/mkr3d/grid-apps /home/semartin/.config/mkr3d/km_slicer.json /home/semartin/.config/mkr3d/km_slicer.json /home/semartin/Downloads/spincaster_display_cover.stl /home/semartin/Downloads/spincaster_display_cover.gcode
{
  msg: 'failed to load',
  dir: '/home/semartin/Documents/devenv/mkr3d/grid-apps',
  file: '/home/semartin/Documents/devenv/mkr3d/grid-apps/src/geo/debug.js'
}
{ fake_worker: '/code/worker.js?3.0.5' }
kiri | init work | 3.0.5
[
  'driver',     'loader',    'license',
  'version',    'copyright', 'conf',
  'work',       'client',    'newEngine',
  'api',        'Top',       'Slice',
  'newTop',     'newSlice',  'slicer',
  'slicer2',    'Layers',    'fill',
  'fill_fixed', 'Widget',    'newWidget',
  'Print',    